## Medical data analysis using graphs

In [1]:
from main import *
#import dataframe_image as dfi

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

studyId = 'nsclc_ctdx_msk_2022'

#### Getting data from local files (downloaded from https://www.cbioportal.org/)

Patient data

In [2]:
patient_data = get_PatientData(studyId)
display(patient_data)

,PATIENT_ID,PATIENT_DISPLAY_NAME,HISTOLOGY,PRIOR_TREATMENT,STAGE_AT_DRAW,AGE_CURRENT,SEX,OS_STATUS,OS_MONTHS,RACE,ETHNICITY,SUCCESSFUL_CTDX_LUNG,SITE,AGE_GT_MEDIAN,SMOKING_STATUS,EXTRAPULMONARY,METABOLIC_TUMOR_VOL
0,MSK-L-001,MSK-L-001,Squamous Cell Carcinoma,True,4.0,NaN,Male,1:DECEASED,14.586071,NaN,NaN,True,MSK,False,True,NaN,NaN
1,P-0016223,MSK-L-002,Other,True,4.0,51.0,Male,0:LIVING,65.867280,ASIAN-FAR EAST/INDIAN SUBCONT,Non-Spanish; Non-Hispanic,True,MSK,False,True,True,NaN
2,MSK-L-003,MSK-L-003,Adenocarcinoma,True,4.0,NaN,Male,0:LIVING,55.551905,NaN,NaN,True,MSK,False,True,NaN,NaN
3,P-0017832,MSK-L-004,Adenocarcinoma,True,4.0,64.0,Female,1:DECEASED,7.227332,WHITE,Non-Spanish; Non-Hispanic,True,MSK,False,True,NaN,NaN
4,P-0015636,MSK-L-005,Adenocarcinoma,False,4.0,78.0,Female,1:DECEASED,3.942181,WHITE,Non-Spanish; Non-Hispanic,True,MSK,False,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,MSK-L-838,MSK-L-838,NaN,NaN,NaN,NaN,NaN,0:LIVING,3.482260,NaN,NaN,False,MSK,NaN,NaN,NaN,NaN
1123,MSK-L-1190,MSK-L-1190,NaN,NaN,NaN,NaN,NaN,0:LIVING,3.383706,NaN,NaN,False,MSK,NaN,NaN,NaN,NaN
1124,MSK-L-738,MSK-L-738,NaN,NaN,NaN,NaN,NaN,0:LIVING,10.906702,NaN,NaN,False,MSK,NaN,NaN,NaN,NaN
1125,MSK-L-1029,MSK-L-1029,NaN,NaN,NaN,NaN,NaN,0:LIVING,23.357424,NaN,NaN,False,MSK,NaN,NaN,NaN,NaN


Sample data

In [3]:
sample_data = get_SampleData(studyId)
display(sample_data)

,SAMPLE_ID,PATIENT_ID,ONCOTREE_CODE,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_CLASS,GENE_PANEL,SAMPLE_TYPE,METASTATIC_SITE,PRIMARY_SITE,TUMOR_PURITY,MSI_SCORE,MSI_TYPE,AGE_AT_SEQ_REPORTED_YEARS,TMB_NONSYNONYMOUS
0,MSK-L-001-001A,MSK-L-001,LUSC,Non-Small Cell Lung Cancer,Lung Squamous Cell Carcinoma,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSK-L-002-001B,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSK-L-002-002,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MSK-L-002-003,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MSK-L-002-004,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2616,MSK-L-838-NA,MSK-L-838,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2617,MSK-L-1190-NA,MSK-L-1190,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2618,MSK-L-738-NA,MSK-L-738,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2619,MSK-L-1029-NA,MSK-L-1029,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Mutation data

In [4]:
mutation_data = get_MutationData(studyId)
display(mutation_data)

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,Validation_Status,Mutation_Status,Sequencing_Phase,Sequence_Source,Validation_Method,Score,BAM_File,Sequencer,t_ref_count,t_alt_count,n_ref_count,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Exon_Number,Annotation_Status,Frequency,Hotspot
0,EGFR,1956,MSKCC,GRCh37,7,55242465,55242479,+,inframe_deletion,In_Frame_Del,DEL,GGAATTAAGAGAAGC,GGAATTAAGAGAAGC,-,rs121913421,NaN,P-0027419-T06-IM7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,SOMATIC,NaN,NaN,NaN,MSK-IMPACT,NaN,NaN,491,131,430.0,0.0,ENST00000275493.2:c.2235_2249del,p.Glu746_Ala750del,p.E746_A750del,ENST00000275493,NM_005228.3,745.0,aaGGAATTAAGAGAAGCa/aaa,19/28,SUCCESS,NaN,0
1,SDHA,6389,MSKCC,GRCh37,5,223682,223682,+,"missense_variant,splice_region_variant",Missense_Mutation,SNP,C,C,T,rs369321221,NaN,P-0027419-T06-IM7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,SOMATIC,NaN,NaN,NaN,MSK-IMPACT,NaN,NaN,639,119,295.0,0.0,ENST00000264932.6:c.149C>T,p.Ser50Phe,p.S50F,ENST00000264932,NM_004168.2,50.0,tCc/tTc,2/15,SUCCESS,NaN,0
2,SDHA,6389,MSKCC,GRCh37,5,223664,223664,+,missense_variant,Missense_Mutation,SNP,C,C,T,NaN,NaN,P-0027419-T06-IM7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,SOMATIC,NaN,NaN,NaN,MSK-IMPACT,NaN,NaN,839,152,368.0,0.0,ENST00000264932.6:c.131C>T,p.Ser44Phe,p.S44F,ENST00000264932,NM_004168.2,44.0,tCt/tTt,2/15,SUCCESS,NaN,0
3,SDHA,6389,MSKCC,GRCh37,5,224491,224491,+,missense_variant,Missense_Mutation,SNP,C,C,T,NaN,NaN,P-0027419-T06-IM7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,SOMATIC,NaN,NaN,NaN,MSK-IMPACT,NaN,NaN,242,54,133.0,0.0,ENST00000264932.6:c.167C>T,p.Pro56Leu,p.P56L,ENST00000264932,NM_004168.2,56.0,cCa/cTa,3/15,SUCCESS,NaN,0
4,SDHA,6389,MSKCC,GRCh37,5,224617,224617,+,missense_variant,Missense_Mutation,SNP,C,C,T,NaN,NaN,P-0027419-T06-IM7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,SOMATIC,NaN,NaN,NaN,MSK-IMPACT,NaN,NaN,598,120,264.0,0.0,ENST00000264932.6:c.293C>T,p.Ser98Leu,p.S98L,ENST00000264932,NM_004168.2,98.0,tCa/tTa,3/15,SUCCESS,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9728,MET,4233,NaN,GRCh37,7,116411862,116411898,+,"splice_region_variant,intron_variant",Splice_Region,DEL,TAGCCGTCTTTAACAAGCTCTTTCTTTCTCTCTGTTT,TAGCCGTCTTTAACAAGCTCTTTCTTTCTCTCTGTTT,-,NaN,NaN,MSK-L-029-001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1411,147,NaN,NaN,ENST00000397752.3:c.2888-39_2888-3del,NaN,p.X963_splice,ENST00000397752,NM_000245.2,963.0,NaN,NaN,SUCCESS,0.0943,0
9729,MET,4233,NaN,GRCh37,7,116411862,116411898,+,"splice_region_variant,intron_variant",Splice_Region,DEL,TAGCCGTCTTTAACAAGCTCTTTCTTTCTCTCTGTTT,TAGCCGTCTTTAACAAGCTCTTTCTTTCTCTCTGTTT,-,NaN,NaN,MSK-L-029-002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3903,1011,NaN,NaN,ENST00000397752.3:c.2888-39_2888-3del,NaN,p.X963_splice,ENST00000397752,NM_000245.2,963.0,NaN,NaN,SUCCESS,0.2057,0
9730,EGFR,1956,NaN,GRCh37,7,55242469,55242469,+,synonymous_variant,Silent,SNP,T,T,C,rs397517095,NaN,MSK-L-744-001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1022,19,NaN,NaN,ENST00000275493.2:c.2239T>C,p.Leu747=,p.L747=,ENST00000275493,NM_005228.3,747.0,Tta/Cta,19/28,SUCCESS,0.0171,0
9731,EGFR,1956,NaN,GRCh37,7,55242469,55242469,+,synonymous_variant,Silent,SNP,T,T,C,rs397517095,NaN,NCI-L-081-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6476,95314,NaN,NaN,ENST00000275493.2:c.2239T>C,p.Leu747=,p.L747=,ENST00000275493,NM_005228.3,747.0,Tta/Cta,19/28,

Full data (by merging sample and mutation data)

In [5]:
full_data = get_FullData(sample_data, mutation_data)
display(full_data)

,SAMPLE_ID,PATIENT_ID,ONCOTREE_CODE,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_CLASS,GENE_PANEL,SAMPLE_TYPE,METASTATIC_SITE,PRIMARY_SITE,TUMOR_PURITY,MSI_SCORE,MSI_TYPE,AGE_AT_SEQ_REPORTED_YEARS,TMB_NONSYNONYMOUS,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,Validation_Status,Mutation_Status,Sequencing_Phase,Sequence_Source,Validation_Method,Score,BAM_File,Sequencer,t_ref_count,t_alt_count,n_ref_count,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Exon_Number,Annotation_Status,Frequency,Hotspot,MUTATION
0,MSK-L-002-001B,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,nan,nan,nan,nan,nan,nan,nan,ALK,238,nan,GRCh37,2,29416141,29416141,+,stop_gained,Nonsense_Mutation,SNP,G,G,T,rs753750497,nan,MSK-L-002-001B,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,770,278,nan,nan,ENST00000389048.3:c.4812C>A,p.Tyr1604Ter,p.Y1604*,ENST00000389048,NM_004304.4,1604.0,taC/taA,29/29,SUCCESS,0.265,0,ALK_2_29416141_29416141
1,MSK-L-002-001B,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,nan,nan,nan,nan,nan,nan,nan,TP53,7157,nan,GRCh37,17,7578461,7578461,+,missense_variant,Missense_Mutation,SNP,C,C,A,rs121912654,nan,MSK-L-002-001B,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,994,272,nan,nan,ENST00000269305.4:c.469G>T,p.Val157Phe,p.V157F,ENST00000269305,NM_001126112.2,157.0,Gtc/Ttc,5/11,SUCCESS,0.215,0,TP53_17_7578461_7578461
2,MSK-L-002-002,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,nan,nan,nan,nan,nan,nan,nan,TP53,7157,nan,GRCh37,17,7578461,7578461,+,missense_variant,Missense_Mutation,SNP,C,C,A,rs121912654,nan,MSK-L-002-002,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4984,2248,nan,nan,ENST00000269305.4:c.469G>T,p.Val157Phe,p.V157F,ENST00000269305,NM_001126112.2,157.0,Gtc/Ttc,5/11,SUCCESS,0.3107,0,TP53_17_7578461_7578461
3,MSK-L-002-003,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,nan,nan,nan,nan,nan,nan,nan,ALK,238,nan,GRCh37,2,29416141,29416141,+,stop_gained,Nonsense_Mutation,SNP,G,G,T,rs753750497,nan,MSK-L-002-003,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1863,25,nan,nan,ENST00000389048.3:c.4812C>A,p.Tyr1604Ter,p.Y1604*,ENST00000389048,NM_004304.4,1604.0,taC/taA,29/29,SUCCESS,0.0132,0,ALK_2_29416141_29416141
4,MSK-L-002-003,P-0016223,NSCLC,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,cfDNA,ctDx_lung_panel,Metastasis,nan,nan,nan,nan,nan,nan,nan,TP53,7157,nan,GRCh37,17,7578461,7578461,+,missense_variant,Missense_Mutation,SNP,C,C,A,rs121912654,nan,MSK-L-002-003,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1759,18,nan,nan,ENST00000269305.4:c.469G>T,p.Val157Phe,p.V157F,ENST00000269305,NM_001126112.2,157.0,Gtc/Ttc,5/11,SUCCESS,0.0101,0,TP53_17_7578461_7578461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9633,NCI-L-199-01,NCI-L-199,LUAD,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,cfDNA,ctDx_lung_panel,Metastasis,nan,nan,nan,nan,nan,nan,nan,KEAP1,9817,nan,GRCh37,19,10602781,10602781,+,missense_variant,Missense_Mutation,SNP,G,G,A,nan,nan,NCI-L-199-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,4134,26,nan,nan,ENST00000171111.5:c.797C>T,p.Ala266Val,p.A266V,ENST00000171111,NM

Drugs data (from file geni_farmaci.xls)

In [ ]:
drugs_data = get_DrugsData()
display(drugs_data)

#### Building the green graph and working on it

In [6]:
dip_graph = build_DiPGraph(full_data)
pm_graph = build_PMGraph(full_data, patient_data, sample_data)

In [7]:
diseases = get_DiNodes(dip_graph)
dd = {'Disease' : [], 'Patient count' : []}
for di in diseases:
    pcnt = len(set(m for m in dip_graph.neighbors(di)))
    dd['Disease'].append(di)
    dd['Patient count'].append(pcnt)
ddf = pd.DataFrame(dd)
ddf.sort_values(by='Patient count', ascending=False, inplace=True)
ddf.reset_index(drop=True, inplace=True)
display(ddf)
#dfi.export(ddfi, f'immagini tesi/{studyId}/malattie.png')
    

,Disease,Patient count
0,Lung Adenocarcinoma,802
1,Non-Small Cell Lung Cancer,113
2,Lung Squamous Cell Carcinoma,61
3,Cancer of Unknown Primary,22
4,"Adenocarcinoma, NOS",10
5,Large Cell Neuroendocrine Carcinoma,9
6,Poorly Differentiated Non-Small Cell Lung Cancer,8
7,"Cancer of Unknown Primary, NOS",6
8,Lung Adenosquamous Carcinoma,6
9,Combined Small Cell Lung Carcinoma,6


In [9]:
disease = 'Lung Adenocarcinoma'
mcount = dip_graph.degree(disease)
df = getMutations_fromDisease(dip_graph, pm_graph, disease, mcount)
print(f'{disease} - {mcount} pazienti')
#dis_mutations = dis_mutations.style.set_caption(f'{disease} - {mcount} patients')
display(df)
#dfi.export(df, f'immagini tesi/{studyId}/panadenocarcinoma.png')

Lung Adenocarcinoma - 802 pazienti


,Mutation,Count,Frequency (%)
0,KRAS_12_25398285_25398285,104,13
1,EGFR_7_55259515_55259515,104,13
2,KRAS_12_25398284_25398284,91,11
3,EGFR_7_55242465_55242479,59,7
4,EGFR_7_55249071_55249071,48,6
...,...,...,...
4579,ERBB2_17_37881001_37881002,1,0
4580,KEAP1_19_10602781_10602781,1,0
4581,MET_7_116436016_116436016,1,0
4582,RET_10_43622158_43622158,1,0


#### Clustering

In [ ]:
cluster_attributes = ['CANCER_TYPE_DETAILED', 'SEX', 'OS_STATUS', 'OS_MONTHS']
threshold = 1   # imposta ad un numero decimale tra 0 e 1 (compresi)
clusters = clustering(pm_graph, threshold)

Display general cluster view

In [ ]:
if len(clusters) > 0:
    cluster_view = {'Cluster' : [], 'Numero pazienti' : [], 'Numero mutazioni': []}
    for n, patients in clusters.items():
        cluster_view['Cluster'].append(n)
        cluster_view['Numero pazienti'].append(len(patients))
        leader = patients[0]
        leader_mutations = list(m for m in pm_graph.neighbors(leader))
        cluster_view['Numero mutazioni'].append(len(leader_mutations))
    cluster_view_df = pd.DataFrame(cluster_view)

    total_patients = len(get_PNodes(pm_graph))
    cluster_patients = cluster_view_df['Numero pazienti'].sum()
    ones = total_patients - cluster_patients
    total_clusters = len(clusters) + ones
    cluster_view_df.iloc[:].plot.bar(x='Cluster', rot=0, title=f'Clustering di {total_patients} pazienti - Soglia di similaritá = {threshold}', figsize=(10, 4))
    with pd.option_context('display.max_rows', 10):
        display(cluster_view_df)
    print(f'{total_patients} pazienti\n{total_clusters} cluster totali: {len(clusters)} cluster con piú di un paziente ({cluster_patients} pazienti), {ones} cluster con un paziente')

Display all clusters (or a specific cluster)

In [ ]:
cluster_dfs = {}
cluster_model = {'PATIENT' : []}
for attr in cluster_attributes:
    if nx.get_node_attributes(pm_graph, attr):
        cluster_model[attr] = []

for n, patients in clusters.items():
    cluster_data = copy.deepcopy(cluster_model)
    for p in patients:
        for attr in cluster_data.keys():
            if attr == 'PATIENT':
                cluster_data[attr].append(p)
            else:
                cluster_data[attr].append(pm_graph.nodes[p][attr])
    cluster_df = pd.DataFrame(cluster_data)
    cluster_dfs[n] = cluster_df

#dfi.export(cluster_dfs[0], f'immagini tesi/{studyId}/cl0.png')

def show_cluster(number):
    #show = 0    # imposta al numero di cluster che si vuole approfondire o a -1 per vederli tutti
    for n, cluster in cluster_dfs.items():
        if n == number or number == -1:
            cluster_df = cluster.style.set_caption(f'Cluster {n}')
            leader = clusters[n][0]
            leader_mutations = list(m for m in pm_graph.neighbors(leader))
            cluster_mutations = {'MUTATION' : leader_mutations}
            cluster_mutations_df = pd.DataFrame(cluster_mutations)
            #dfi.export(cluster_mutations_df, f'immagini tesi/{studyId}/cl0_mutations.png')
            display(cluster_df)
            display(cluster_mutations_df)
            '''for attr in cluster_attributes:
                if attr == 'OS_MONTHS':
                    cluster[attr].plot.bar(xlabel='Patient', rot=0, title=attr)
                else:
                    cluster[attr].value_counts().plot(kind='pie', autopct='%1.0f%%', title=attr, ylabel='')
                plt.show()'''
            #cluster[attribute].value_counts().plot(kind='pie', autopct='%1.0f%%', title=attribute)

show_cluster(-1)